<a href="https://colab.research.google.com/github/olaviinha/MidiTurmoil/blob/main/SequencePatternGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Sequence Pattern Generator<font color="#999" size="3">&nbsp;&nbsp;&nbsp;&nbsp;v0.0.6</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/MidiTurmoil" target="_blank"><font color="#999" size="4">Github</font></a></font>

Sequence Pattern Generator generates randomized sequences based on given chord progression (or just individual chord) and other settings, and outputs a MIDI file.

- `output_dir` (optional) should be relative to your Google Drive root (My Drive). E.g. `/music/Midi` if you have a directory called _music_ in your Drive, containing a subdirectory called _Midi_. Paths are case-sensitive.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will setup prerequisites.<br>
#@markdown <small>Mounting Drive will enable this notebook to auto-save outputs directly to your Drive.</small>

force_setup = False
repositories = ['https://github.com/olaviinha/MidiTurmoil']
pip_packages = 'mido pychord pyfluidsynth midi2audio'
apt_packages = 'fluidsynth'
mount_drive = False #@param {type:"boolean"}
skip_setup = False #@ param {type:"boolean"}

# Download the repo from Github
import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  if apt_packages != '':
    !apt-get update && apt-get install {apt_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if mount_drive == True:
  if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = '/content/drive/My Drive'
  if not os.path.isdir('/content/mydrive'):
    os.symlink('/content/drive/My Drive', '/content/mydrive')
    drive_root = '/content/mydrive/'
  drive_root_set = True
else:
  create_dirs(['/content/faux_drive'])
  drive_root = '/content/faux_drive/'

if len(repositories) > 0 and skip_setup == False:
  for repo in repositories:
    %cd /content/
    install_dir = fix_path('/content/'+path_leaf(repo).replace('.git', ''))
    repo = repo if '.git' in repo else repo+'.git'
    !git clone {repo}
    if os.path.isfile(install_dir+'setup.py') or os.path.isfile(install_dir+'setup.cfg'):
      !pip install -e ./{install_dir}
    if os.path.isfile(install_dir+'requirements.txt'):
      !pip install -r {install_dir}/requirements.txt

if len(repositories) == 1:
  %cd {install_dir}

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])

import time, sys
from datetime import timedelta
import math















if not os.path.isdir('/content/sf') and force_setup == False:
  !gsutil -q -m cp -R gs://neural-research/olaviinha/sf/* {dir_tmp}

from random import randrange
from mido import Message, MidiFile, MidiTrack, MetaMessage
import mido

import random
from itertools import groupby
from pychord import Chord

from midi2audio import FluidSynth
from IPython.display import display, Audio
import numpy as np
import matplotlib.pyplot as plt
import roll



debug = False

def generate_notes(chord='Amaj', octave=3, additional_octaves=0, notes_per_beat=1, beats_per_chord=4, repeat_every=0):
  global debug
  chord_notes = Chord(chord).components_with_pitch(root_pitch=octave)

  if additional_octaves > 0:
    for i in range(additional_octaves):
      if i > 0:
        chord_notes.extend(Chord(chord).components_with_pitch(root_pitch=octave+i))

  notes = chord_notes.copy()
  random.shuffle(notes)
  all_notes = []

  if repeat_every > 0:
    for _ in range(repeat_every):
      extended_notes = []
      for _ in range(8):
        extended_notes.extend(random.sample(notes.copy(), len(notes)))
      nrpn = [i[0] for i in groupby(extended_notes)]
      beat_notes = nrpn[:notes_per_beat]
      all_notes.extend(beat_notes)
  else:
    for _ in range(beats_per_chord):
      extended_notes = []
      for _ in range(8):
        extended_notes.extend(random.sample(notes.copy(), len(notes)))
      if debug == True: print('extended_notes:', extended_notes)

      nrpn = [i[0] for i in groupby(extended_notes)]
      beat_notes = nrpn[:notes_per_beat]
      if debug == True: print('beat_notes:', beat_notes)
      all_notes.extend(beat_notes)

  if debug == True: print('\nall_notes:', all_notes, '')

  #remove repeats from all_notes
  nrpn = [i[0] for i in groupby(all_notes.copy()*8)]
  final_notes = nrpn[:notes_per_beat*beats_per_chord]

  return final_notes


def notes2midi(all_notes):
  midi_notes = []
  for note in all_notes:
    midi_note = librosa.note_to_midi(note)
    midi_notes.append(midi_note)
  if debug == True: print(midi_notes)
  return midi_notes


def plot_notation(midifile):
  import roll
  midr = roll.mfplt(midifile)
  midr.draw_roll()

def get_differences(blist, round=0):
  #x = list(np.array(blist.tolist()).flatten())
  x = blist
  xdiff = [x[n]-x[n-1] for n in range(1,len(x))]
  if round > 0:
    rounded = [ '%.2f' % el for el in xdiff ]
    return rounded
  else:
    return xdiff

def bouncer(gravity=-8.87, direction='out', bounce_length=10000):
  import numpy as np
  import matplotlib.pyplot as plt
  g=gravity
  #t=np.linspace(0,4500,bounce_length,dtype=np.float64)
  t=np.linspace(0, bounce_length, bounce_length, dtype=np.float64)
  v=np.zeros(bounce_length)
  y=np.zeros(bounce_length)
  bounces=0
  y[0]=bounce_length*(bounce_length/100)
  for i in range(1, bounce_length):
    if y[i-1]+0.9*v[i-1]>0:
      v[i]=v[i-1]+g*(0.9)
      y[i]=y[i-1]+0.9*v[i-1]
    if y[i-1]+0.9*v[i-1]<=0:
      v[i]=-0.9*v[i-1]
      y[i]=0
      bounces+=1
  if direction == 'in':
    y = np.flip(y)
  return y

output.clear()
op(c.ok, 'Setup finished.')


In [ ]:
#@markdown # Base settings
chord_progression = "E Am D G" #@param {type: "string"}
bpm = 126 #@param {type:"slider", min:60, max:200, step:1}
root_octave = 3 #@param {type:"slider", min:1, max:8, step:1}
#@markdown <small>All additional octaves will be  _above_ root octave.</small>
octaves = 3 #@param {type:"slider", min:1, max:4, step:1}

#@markdown ### Note hits
velocity_min = 76 #@param {type:"slider", min:1, max:127, step:1}
velocity_max = 111 #@param {type:"slider", min:0, max:127, step:1}

#@markdown ### Output
preview_instrument = "jazz_guitar" #@param ["piano", "jazz_guitar", "organ", "rhodes", "tubularbell", "303", "sine_saw", "NES"]
#@markdown <small>If you fill `output_dir` (directory path), all generated MIDI files will be automatically saved there.</small>
output_dir = "" #@param {type:"string"}
auto_download = False #@ param {type:"boolean"}

og_output_dir = output_dir

sf2dir = dir_tmp+'base/'
if preview_instrument == 'piano': pi = sf2dir+'SalC5Light2.sf2'
if preview_instrument == 'jazz_guitar': pi = sf2dir+'inha-FatBoy-jazzguitar.sf2'
if preview_instrument == 'organ': pi = sf2dir+'inha-FatBoy-organ-rock.sf2'
if preview_instrument == 'rhodes': pi = sf2dir+'inha-FatBoy-rhodes.sf2'
if preview_instrument == 'tubularbell': pi = sf2dir+'inha-FatBoy-tubularbell.sf2'
if preview_instrument == '303': pi = sf2dir+'inha-HSTB303.sf2'
if preview_instrument == 'sine_saw': pi = sf2dir+'inha-nes-sinesaw.sf2'
if preview_instrument == 'NES': pi = sf2dir+'inha-nes-base.sf2'

# if quality == '<none>':
#   quality = ''

if output_dir != '':
  output_dir = fix_path(drive_root+output_dir)
  if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
else:
  output_dir = dir_tmp

chord_prog_chords = [x.strip() for x in chord_progression.split(' ')]

errors = []
for cp_chord in chord_prog_chords:
  try:
    Chord(cp_chord)
  except:
    errors.append(cp_chord)

# output.clear()

# octaves = octaves - 1 

if len(errors) > 0:
  op(c.fail, 'ERROR! Please check your chord progression:')
  for error in errors:
    print('-', error, 'is not a valid chord!')
    sys.exit('Check chords')

uniq_id = gen_id()

op(c.title, 'ID:', uniq_id, time=True)
op(c.ok, 'Done.', time=True)

In [ ]:
#@markdown # Generate pattern

# scale = 'E' #@param ["Ab", "A", "A#", "Bb", "B", "Cb", "C", "C#", "Db", "D", "D#", "Eb", "E", "F", "F#", "Gb", "G", "G#"]
# quality = '9sus4' #@param ["---- 2 notes ----", "5", "sus", "---- 3 notes ----", "<none>", "maj", "m", "min", "-", "dim", "aug", "sus2", "sus4", "---- 4 notes ----", "6", "7", "7-5", "7b5", "7+5", "7#5", "7sus4", "m6", "m7", "m7-5", "m7b5", "dim6", "M7", "maj7", "M7+5", "mM7", "add9", "madd9", "2", "add11", "4", "---- 5 notes ----", "m69", "69", "9", "m9", "M9", "maj9", "9sus4", "7-9", "7b9", "7+9", "7#9", "9-5", "9b5", "9+5", "9#5", "7#9b5", "7#9#5", "7b9b5", "7b9#5", "11", "7+11", "7#11", "7b9#9", "7b9#11", "7#9#11", "7-13", "7b13", "---- 6 notes ----", "7b9b13", "9+11", "9#11", "13", "13-9", "13b9", "13+9", "13#9", "13+11", "13#11"]

notes_per_beat = 4 #@param {type:"slider", min:1, max:16, step:1}
beats_per_chord = 4 #@param {type:"slider", min:1, max:16, step:1}
bar_repetition = 0 #@param {type:"slider", min:0, max:4, step:1}
batch = 1 #@param {type:"integer"}
# #@markdown <small>Output length in bars of 4 beats.</small>
# output_length = 16 #@param {type:"slider", min:4, max:64, step:4}

output_length = beats_per_chord
if batch == 0: batch = 1

for b in range(batch):
  key = rnd_str(4)
  chords_notes = []
  for cp_chord in chord_prog_chords:
    notes = generate_notes(
              chord=cp_chord, 
              octave=root_octave, 
              additional_octaves=octaves, 
              notes_per_beat=notes_per_beat, 
              beats_per_chord=beats_per_chord,
              repeat_every=bar_repetition
            )
    midi_notes = notes2midi(notes)
    chords_notes.append(midi_notes)


  ticks_per_beat = 128
  note_time = math.floor(ticks_per_beat/notes_per_beat)

  mid = MidiFile(ticks_per_beat=ticks_per_beat)
  track = MidiTrack()
  mid.tracks.append(track)

  track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
  track.append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(bpm), time=0))

  for midi_notes in chords_notes:
    for i, midi_note in enumerate(midi_notes):
      velocity = randrange(velocity_min, velocity_max)
      track.append(Message('note_on', note=midi_note, velocity=velocity, time=0))
      track.append(Message('note_off', note=midi_note, velocity=127, time=note_time))

  if octaves == 1:
    octo = '_oct'+str(root_octave)
  else:
    octo = '_oct'+str(root_octave)+'-'+str(root_octave+octaves-1)

  fn_chords = '_'.join([x.replace('/', '-') for x in chord_prog_chords])
  midi_filename = output_dir+uniq_id+'_'+str(b+1).zfill(2)+'_pattern_'+str(bpm)+'bpm_'+fn_chords+'__'+key+'.mid'
  mid.save(midi_filename)

  if os.path.isfile(midi_filename):
    plt.rcParams['figure.figsize'] = [25, 6]
    plt.rcParams.update({"axes.facecolor": "black"})
    audio_preview = 'output.wav'
    FluidSynth(pi).midi_to_audio(midi_filename, audio_preview)
    if batch == 1:
      plot_notation(midi_filename)
    audio_player(audio_preview)
    
    if og_output_dir != '':
      op(c.ok, 'File saved as:', midi_filename.replace(drive_root, ''))
    else:
      if auto_download == True: 
        files.download(midi_filename)
      else:
        dl_btn(midi_filename)
  else:
    op(c.fail, 'ERROR saving MIDI', midi_filename.replace(drive_root, ''))

In [ ]:
#@markdown # Generate bounce(s)

#@markdown <small>Individual MIDI file will be generated for each chord.</small>

build_up = 3200 #@param {type:"slider", min:100, max:5000, step:100}
fall_down = 9700 #@param {type:"slider", min:1000, max:20000, step:100}
show_bounces = False
indices = []
diffs = []
mid = ''
track = ''
midr = ''
roll = ''

shave_heads = 0
shave_tails = 0

key = rnd_str(4)

chords_notes = []
for cp_chord in chord_prog_chords:
  notes = generate_notes(
            chord=cp_chord, 
            octave=root_octave, 
            additional_octaves=octaves, 
            notes_per_beat=notes_per_beat, 
            beats_per_chord=beats_per_chord
          )
  midi_notes = notes2midi(notes)
  chords_notes.append(midi_notes)

# midi_notes = notes2midi(generate_notes(chord=scale+quality, octave=root_octave, additional_octaves=octaves))

total_bounces = len(chords_notes)

for chord_index, midi_notes in enumerate(chords_notes, 0):

  op(c.title, 'Bouce for chord:', chord_prog_chords[chord_index])
  print()
  plt.rcParams['figure.figsize'] = [25, 6]
  plt.rcParams.update({"axes.facecolor": "black"})

  bounce_in = bouncer(-4.87, 'out', build_up)
  if show_bounces == True:
    plt.plot(bounce_in)
    plt.show()

  bounce_out = bouncer(-8.87, 'in', fall_down)
  if show_bounces == True:
    plt.plot(bounce_out)
    plt.show()

  full_bounce = np.concatenate((bounce_in, bounce_out))
  if show_bounces == True:
    plt.plot(full_bounce)
    plt.show()

  indices = [i for i, x in enumerate(full_bounce) if x == 0]
  diffs = get_differences(indices)
  diffs = get_differences(indices)
  diffs = [x[0] for x in groupby(diffs)] + [diffs[-1] ] * 10

  ticks_per_beat = 128
  note_time = math.floor(ticks_per_beat/notes_per_beat)
  mid = MidiFile(ticks_per_beat=ticks_per_beat)
  track = MidiTrack()
  mid.tracks.append(track)

  track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
  track.append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(bpm), time=0))

  midi_notes = midi_notes * 50
  for i, hit in enumerate(diffs):
    if i < len(diffs)-1:

      note_time = math.floor(hit/10)
      velocity = randrange(velocity_min, velocity_max)
      track.append(Message('note_on', note=midi_notes[i], velocity=velocity, time=0))
      track.append(Message('note_off', note=midi_notes[i], velocity=127, time=note_time))

  if octaves == 1:
    octo = '_oct'+str(root_octave)
  else:
    octo = '_oct'+str(root_octave)+'-'+str(root_octave+octaves-1)

  # midi_filename = output_dir+uniq_id+'_bounce_'+str(bpm)+'bpm_'+str(i).zfill(2)+'_'+chord_prog_chords[chord_index]+'__'+key+'.mid'

  fn_chords = '_'.join([x.replace('/', '-') for x in chord_prog_chords])
  midi_filename = output_dir+uniq_id+'_'+str(chord_index+1).zfill(2)+'_bounce_'+str(bpm)+'bpm_'+fn_chords+'__'+key+'.mid'

  mid.save(midi_filename)

  if os.path.isfile(midi_filename):
    plt.rcParams['figure.figsize'] = [25, 6]
    plt.rcParams.update({"axes.facecolor": "black"})
    audio_preview = 'output.wav'
    FluidSynth(pi).midi_to_audio(midi_filename, audio_preview)
    if total_bounces == 1:
      plot_notation(midi_filename)
    audio_player(audio_preview)
    if og_output_dir != '':
      op(c.ok, 'File saved as:', midi_filename.replace(drive_root, ''))
    else:
      if auto_download == True: 
        files.download(midi_filename)
      else:
        dl_btn(midi_filename)
  else:
    op(c.fail, 'ERROR saving MIDI', midi_filename.replace(drive_root, ''))
